# Medical Appointment No Shows
This analysis is part of the Udacity Data Analysis Nanodegree program and aims to explore a dataset containing aproximately 100k medical appointments from the Brazilian public health system. 

## Table of Contents
<ul>
<li><a href="#intro">1. Introduction</a></li>
<li><a href="#wrangling">2. Data Wrangling</a></li>
<li><a href="#eda">3. Exploratory Data Analysis</a></li>
<li><a href="#conclusions">4. Conclusions</a></li>
<li><a href="#references">References</a></li>
</ul>

<a id='intro'></a>
## 1. Introduction

The Brazilian public health system, known as SUS for *Unified Health System* in its acronym in Portuguese, is one of the largest health system in the world <a href="#[1]">[1]</a>, representing government investment of more than 9% of GDP. However, its operation is not homogeneous and there are distinct perceptions of quality from citizens in different regions of the country.

This analysis will use the *no-show appointments dataset* which collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment, with a number of characteristics about the patient included in each row:
- ‘ScheduledDay’: tells us on what day the patient set up their appointment.  
- ‘Neighborhood’: indicates the location of the hospital.  
- ‘Scholarship’: indicates whether or not the patient is enrolled in Brasilian welfare program called *Bolsa Família*.
- 'No-show': it says ‘No’ if the patient showed up to their appointment, and ‘Yes’ if they did not show up.

This analysis aims to outline the reasons for patient no-showing to scheduled appointments, as well as get insights about the Brazilian public health system. To accomplish this, we will first try to understand the data and the context on which they were collected. We will then proceed to data wrangling and exploratory data analysis, in an iterative process, in order to draw some conclusions about the subject.

### 1.1. Initial statements
This section sets up import statements for all the packages that will be used throughout this notebook.

In [4]:
# Data analysis packages:
import pandas as pd
import numpy as np

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### 1.2. Understanding the data
Since the original dataset is available at Kaggle <a href="#[2]">[2]</a>, the first step was to get additional information available on its original repository, as transcripted below:

>In response here it goes: Country? Brazil Is this from private or public service? Or both? Public Sector (Brazil) Primary, secondary, or tertiary care? Primary care, not sure about it. Only physicians? What would be the specialties? We don't have these information.
  
>First appointment with that specific professional? --> I believe first appointments are prone to "no show". That is a very good hint, we don't have that information yet. Part of the day? --> Early morning might be worse. (Perhaps in the next version we can include it.

Another relevant point is that any information is provided about the source of data. I.e., did these data come from a government agency or were they a compilation from each health unit? Furthermore, besides it is known the data is about public health unit, it is not said from which government sphere they belong to (municipal, state or federal). 
The absence of such information can result in a biased analysis, since the complex and decentralized Brazilian health system brings inevitable conflicts and contradictions between its associated entities, as identified in <a href="#[3]">[3]</a>.

In order to define the hypotheses that will lead this analysis, it is first needed to know which information is available in the dataset (attributes). Thus, the first lines of the dataset are shown below:

In [9]:
pd.read_csv('noshowappointments-kagglev2-may-2016.csv').head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


### 1.3. Hypotheses to be explored

From the dataset information and its attributes, the following questions can be formulated:
* **Regarding not showing to appointment**:
  - Do the patients forget the appointment?
  - What is the average waiting time between the scheduling date and the appointment date?
  - What is the *missing appointments per patient* relation in the dataset?  
  - How many patients have missed an appointment at least once?
  
* **Regarding the patient profile**:
  - Is there any common characteristics among those patients that miss appointments?
  - Patients suffering from serious illnesses are more assiduous?

* **Regarding the health unit**:
  - What are the neighbourhoods with the highest no-showing rate?
  - Considering there are common characteristics among those who do not attend the appointments, how these characteristics are geographically distributed?
  
The analysis conduced in the next sections will try to address these questions and to answer them based on the available dataset.

<a id='wrangling'></a>
## 2. Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### 2.1. General Properties

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.


In [7]:
dataset = pd.read_csv('noshowappointments-kagglev2-may-2016.csv')
dataset.head(3)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [5]:
dataset['Neighbourhood'].unique()

array(['JARDIM DA PENHA', 'MATA DA PRAIA', 'PONTAL DE CAMBURI',
       'REPÚBLICA', 'GOIABEIRAS', 'ANDORINHAS', 'CONQUISTA',
       'NOVA PALESTINA', 'DA PENHA', 'TABUAZEIRO', 'BENTO FERREIRA',
       'SÃO PEDRO', 'SANTA MARTHA', 'SÃO CRISTÓVÃO', 'MARUÍPE',
       'GRANDE VITÓRIA', 'SÃO BENEDITO', 'ILHA DAS CAIEIRAS',
       'SANTO ANDRÉ', 'SOLON BORGES', 'BONFIM', 'JARDIM CAMBURI',
       'MARIA ORTIZ', 'JABOUR', 'ANTÔNIO HONÓRIO', 'RESISTÊNCIA',
       'ILHA DE SANTA MARIA', 'JUCUTUQUARA', 'MONTE BELO',
       'MÁRIO CYPRESTE', 'SANTO ANTÔNIO', 'BELA VISTA', 'PRAIA DO SUÁ',
       'SANTA HELENA', 'ITARARÉ', 'INHANGUETÁ', 'UNIVERSITÁRIO',
       'SÃO JOSÉ', 'REDENÇÃO', 'SANTA CLARA', 'CENTRO', 'PARQUE MOSCOSO',
       'DO MOSCOSO', 'SANTOS DUMONT', 'CARATOÍRA', 'ARIOVALDO FAVALESSA',
       'ILHA DO FRADE', 'GURIGICA', 'JOANA D´ARC', 'CONSOLAÇÃO',
       'PRAIA DO CANTO', 'BOA VISTA', 'MORADA DE CAMBURI', 'SANTA LUÍZA',
       'SANTA LÚCIA', 'BARRO VERMELHO', 'ESTRELINHA', 'FORTE SÃO 

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (Replace this with more specific notes!)

In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!

<a id='references'></a>
## References

<a id='[1]'></a>[1] Wikipedia contributors. Brazil. Wikipedia, The Free Encyclopedia. February 9, 2018, 22:06 UTC. Available at: https://en.wikipedia.org/w/index.php?title=Brazil&oldid=824851232. Accessed February 10, 2018.

<a id='[2]'></a>[2] Kaggle. *Medical Appointment No Shows*. Reviewed dataset. Available at: https://www.kaggle.com/joniarroba/noshowappointments. Accessed February 10, 2018.

<a id='[3]'></a>[3] Victora, C. G., Barreto, M. L., Leal, M. D. C., Monteiro, C. A., Schmidt, M. I., Paim, J. S., ... & Reichenheim, M. E. (2011). *Condições de saúde e inovações nas políticas de saúde no Brasil: o caminho a percorrer* (in Portuguese). Available at: https://repositorio.ufba.br/ri/handle/ri/5973. Accessed February 12, 2018.

### Backlog (Verificar a utilidade disso na análise)

> Aqui o ponto principal será identificar de onde são esses dados. Das informações disponíveis no Kagle, já sabe-se que são de Vitória, ES. Portanto, vai ser muito interessante cruzar com os dados demográficos dessa cidade, a partir do IBGE.
>

##### About the Introduction Section
> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.